# init

In [26]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

from Geometry import *
from camera import Camera
import draw
from inputs import input_update

# reload modules

In [51]:

importlib.reload(Clipping)
importlib.reload(sys.modules['Geometry'])
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
from Geometry import *

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [57]:
from colors import *
    
d = 4 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = build_cube(d)


if d == 4:
    for face, color in zip(cube_shape.faces,tess_face_colors):
        face.color = color
    shapes = []
    for i in range(4):
        cube = cube_shape.copy()
        cube.update(pos = 2*np.eye(d)[i])
        shapes.append(cube)
        cube = cube_shape.copy()
        cube.update(pos = -2*np.eye(d)[i])
        shapes.append(cube)
    
    #shapes = [cube_shape] #single cube
    #cube_shape.transparent = True
if d == 3:
    for face, color in zip(cube_shape.faces,cube_face_colors):
        face.color = color
    cube1 = cube_shape.copy()
    cube1.update(pos=np.array([2,0,0]))

    cube2 = cube_shape.copy()
    cube2.update(pos=np.array([-2,0,0]))

    cube3 = cube_shape.copy()
    cube3.update(pos=np.array([0,0,2]))

    cube4 = cube_shape.copy()
    cube4.update(pos=np.array([0,0,-2]))

    cube5 = cube_shape.copy()
    cube5.update(pos=np.array([0,-2,0]))
    
    shapes = [cube1,cube2,cube3,cube4,cube5]
    shapes[0].transparent = True
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Does clipping happen in 2 or 3? Where should it happen?

In [58]:

t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

#default arguments are annoying for inheritance.
#this is why draw_origin and focal are specified here
if d == 3:
    draw.init(d,size,focal=6)
    draw.view_radius = 7

    camera = Camera(pos = Point([0.8,2,0.8]))
    camera.update_rot_matrix(0,-1,-3*np.pi/4)
    camera.update_rot_matrix(1,-1,-np.pi/4)

if d == 4:
    draw.init(d,size,stereo=True)
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
    draw.face_scales = [0.95,0.5]
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    camera = Camera(pos = Point([0,0,0,0]))
    #camera.look_at(np.array([2,2,2,2]))
    #camera = Camera(pos = np.array([0.8,2,0.8,0.8]),draw_class = draw_class)
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
draw.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes)
            pygame.display.flip()
            redraw = False

        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, draw, shapes) 
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL
70.25869378725237 fps
38.730461018117225 fps
27.515563715742317 fps
23.383001479969174 fps
23.108676807107912 fps
23.3953754764054 fps
23.52916196408543 fps
23.541107951014304 fps
17.25724145608633 fps
16.31103284171943 fps
17.096864846961015 fps
15.188699493968661 fps
26.958241535364326 fps
51.05936188874106 fps
44.45910038022587 fps
26.291218102704093 fps
22.397149702492303 fps
14.120351992982707 fps
13.392814162775032 fps
10.747384309010444 fps
10.337074716843667 fps
10.35339310174104 fps
10.243214103755816 fps
10.237312610226715 fps
10.330375169958442 fps
10.343593464728183 fps
10.236432195902703 fps
10.340259014845744 fps
10.256322404831629 fps
10.243713250780777 fps
10.380610593621492 fps
12.292713967696129 fps
13.499205787161534 fps
11.232455814023892 fps
13.012308044794173 fps
12.047256750656635 fps
11.934222842084909 fps
11.459661853883093 fps
11.777701467156838 fps
11.989883375819439 fps
11.834370325365166 fps
11.867317221771158 fps
11.968946914595454 fps
game termina

In [24]:
pygame.quit()

In [35]:
draw.stereo

False